# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Этап 1. Получение данных

Импортируем библиотеку pandas, прочитаем файл data.csv и сохраним его в переменную data.
Получим первые 10 строк.

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице, а также проверим наличие пропусков.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим, что количество значений в стобцах различается, значит есть отсутствующие значения.
Отсутствующие значения имеются в столбцах days_employed и total_income.

Помимо этого, видим, что столбец days_employed имеет вещественный тип данных, что странно, так как количество дней должно быть целым, а также отрицательные значения.

При первом взгляде на таблицу также бросается в глаза, что в столбце education имеются одинаковые по сути значения, но отличающиеся регистром, а в столбце purpose - отличающиеся формой записи.
Просмотрим эти столбцы на уникальность значений и убедимся в этом.

In [ ]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Проверим на уникальность также столбцы dob_years, family_status, children.

In [ ]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

В столбце возраста есть нулевые значения.

In [ ]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В этом столбце со значениями все в порядке.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце с количеством детей имеются отрицательные значения, а также артефакт - 20 детей.

**Вывод**

- Проанализировав данные, нашли, что имеются отсутствующие значения в стобцах days_employed и total_income. Значения NaN, предположительно, это не работающие в данный момент люди, поэтому и не имеющие дохода.<br>
- Столбец days_employed к тому же имеет вещественный тип и содержит отрицательные значения.<br>
- Столбец education имеет одинаковые значения, но отличающиеся регистром.<br>
- Столбец purpose тоже имеет одинаковые значения, но отличающиеся формой записи. Значения в этих двух столбцах скорее всего заполняются вручную, что и приводит к таким разнящимся значениям.<br>
- Столбец dob_years содержит нулевые значения.<br>
- Столбец children имеет одно отрицательное значение, а также одно значение очень выбивающееся из выгрузки - 20 детей.<br>
Надо привести все в порядок: заполнить отсутствующие значения, привести значения к нужному типу данных, избавиться от дубликатов.<br>

## Этап 2. Предобработка данных

### Обработка пропусков

Обрабатывая пропуски в столбцах days_employed и total_income, а также нулевые значени в столбце dob_years, определим, что они имеют количественный тип. Заполним пропуски в этих переменных характерными значениями.

#### Обработка пропусков в столбце total_income

Посмотрим на строки, имеющие пропуски в столбце total_income.

In [ ]:
data[data['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Подсчитаем количество пропусков в столбце total_income по каждому типу занятости клиентов. Так увидим те значения income_type, у которых пропущены значения в столбце total_income.

In [ ]:
data.loc[data['total_income'].isnull(), 'income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Подсчитаем медианный доход по каждому типу занятости и запишем значения в таблицу data_total_income_median.

In [ ]:
data_total_income_median = data.groupby('income_type')['total_income'].median()
data_total_income_median

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполним пропуски в столбце total_income медианными значеними ежемесячного дохода по каждому типу занятости. Проверим, остались ли пропуски в этом столбце.

In [ ]:
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = data_total_income_median[6]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = data_total_income_median[3]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = data_total_income_median[4]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = data_total_income_median[2]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = data_total_income_median[2]
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

#### Обработка нулевых значений в столбце dob_years

Подсчитаем нулевых значений в столбце dob_years по каждому типу занятости клиентов. Так увидим те значения income_type, у которых в столбце dob_years стоит 0.

In [ ]:
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

Подсчитаем медианный возраст по каждому типу занятости клиентов запишем значения в таблицу data_dob_years_median.

In [ ]:
data_dob_years_median = data.groupby('income_type')['dob_years'].median()
data_dob_years_median

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Заполним пропуски в столбце dob_years медианными значеними ежемесячного дохода по каждому типу занятости. Проверим, остались ли пропуски в этом столбце.

In [ ]:
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = data_dob_years_median[6]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = data_dob_years_median[3]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = data_dob_years_median[4]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = data_dob_years_median[2]
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

#### Обработка пропусков в столбце days_employed

Заполним пропуски в столбце days_employed медианными значениями общего трудового стажа по каждому возрасту клиента. Для этого сначала рассчитаем эти медианные значения.

In [ ]:
data_days_employed_median = data.groupby('dob_years')['days_employed'].median()
data_days_employed_median.head()

dob_years
19.0   -724.492610
20.0   -674.838979
21.0   -618.733817
22.0   -698.101581
23.0   -690.204208
Name: days_employed, dtype: float64

Подсчитаем количество пропусков в столбце days_employed по каждому возрасту клиента.

In [ ]:
data.loc[data['days_employed'].isnull(), 'dob_years'].value_counts()

34.0    69
40.0    66
42.0    65
31.0    65
35.0    64
36.0    63
47.0    59
41.0    59
30.0    58
39.0    58
28.0    57
57.0    56
58.0    56
54.0    55
38.0    54
56.0    54
52.0    53
37.0    53
33.0    51
50.0    51
45.0    50
51.0    50
29.0    50
49.0    50
43.0    50
55.0    48
46.0    48
48.0    46
44.0    44
53.0    44
60.0    42
62.0    38
61.0    38
64.0    37
32.0    37
27.0    36
23.0    36
26.0    35
59.0    34
63.0    29
25.0    23
24.0    21
65.0    20
66.0    20
21.0    18
22.0    17
67.0    16
68.0     9
20.0     5
69.0     5
71.0     5
70.0     3
72.0     2
19.0     1
73.0     1
Name: dob_years, dtype: int64

Для того, чтобы не писать длинный код замены пропусков в столбце days_employed медианными значениями по каждому возрасту, по которому эти пропуски встречаются, распределим клиентов на возрастные группы и рассчитаем медианные значения уже по каждой такой группе.

In [ ]:
def age_group(dob_years):
    
        """
        Возвращает возрастную группу по значению возраста age, используя правила:
        - 'молодой' при значении dob_years менее или равно 25 лет
        - 'взрослый' при значениии dob_years более 25 и менее 50, включая 50
        - 'пожилой' при значениии dob_years более 50
        """
        
        if dob_years <= 25:
                return 'молодой'
        if dob_years <= 60:
                return 'взрослый'
        return 'пожилой'

    
data['age_group'] = data['dob_years'].apply(age_group)
data_days_employed_age_group_median = data.groupby('age_group')['days_employed'].median()
data_days_employed_age_group_median

age_group
взрослый     -1452.197294
молодой       -796.983636
пожилой     356191.137667
Name: days_employed, dtype: float64

Подсчитаем количество пропусков в столбце days_employed по каждой возрастной категории.

In [ ]:
data.loc[data['days_employed'].isnull(), 'age_group'].value_counts()

взрослый    1830
пожилой      223
молодой      121
Name: age_group, dtype: int64

Заполним пропуски в столбце days_employed медианными значеними общего трудового стажа по каждой возрастной группе. Проверим, остались ли пропуски в этом столбце.

In [ ]:
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == 'взрослый'),  'days_employed'] = data_days_employed_age_group_median[0]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == 'молодой'), 'days_employed'] = data_days_employed_age_group_median[1]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == 'пожилой'), 'days_employed'] = data_days_employed_age_group_median[2]
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

Переведем все значения столбца days_employed из вещественного в целочисленный тип и все отрицательные значения в положительные. Посмотрим первые 5 строк таблицы.

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')
data['days_employed'] = data['days_employed'].abs()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,взрослый
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослый
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,взрослый
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,взрослый
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослый


Подробнее рассмотрим значения столбца days_employed: переведем медианные значения общего трудового стажа для каждой возрастной группы в года. 

In [ ]:
print('Общий трудовой стаж в годах для возрастной категории "молодой": {} лет'.format(abs(data_days_employed_age_group_median[0] / 365)))
print('Общий трудовой стаж в годах для возрастной категории "взрослый": {} лет'.format(abs(data_days_employed_age_group_median[1] / 365)))  
print('Общий трудовой стаж в годах для возрастной категории "пожилой": {} лет'.format(abs(data_days_employed_age_group_median[2] / 365))) 

Общий трудовой стаж в годах для возрастной категории "молодой": 3.9786227219243355 лет
Общий трудовой стаж в годах для возрастной категории "взрослый": 2.1835168107904472 лет
Общий трудовой стаж в годах для возрастной категории "пожилой": 975.8661305941084 лет


Видим, что помимо того, что столбец days_employed имеет вещественный тип данных, а также содержит отрицательные значения, при переводе его значений в года получаются аномальные большие значения в сотни лет или наоборот всего лишь в пару лет.<br>Тип данных столбца переведен в целочисленный с помощью метода astype() с аргументом int, а отрицательные значения в положительные переведены с помощью функции abs.<br>Однако, прежде всего лучше уточнить у тех, кто сделал выгрузку, по какому принципу рассчитываются значения этого столбца, и только потом обработать их. Предположительно, в этом столбце рассчитывается диапазон дат, возможно, заполненных некорректно.

**Вывод**

Обработаны все пропуски в данных:<br>
- Пропуски в столбце total_income заполнены медианными значеними ежемесячного дохода по каждому типу занятости.<br>
- Нулевые значения в столбце dob_years заполнены медианными значеними ежемесячного дохода по каждому типу занятости.<br>
- Пропуски в столбце days_employed заполнены медианными значеними общего трудового стажа по каждой возрастной группе.

### Замена типа данных

#### Обработка значений столбца children

Переведем все отрицательные значения столбца children в положительные, а также заменим артефакт 20 на 2 из предположения, что заполняющим это значение сотрудником имелось в виду значение 2 ребенка.<br>Выведем уникальные значения столбца children, чтобы убедиться в корректности его значений.

In [ ]:
data['children'] = data['children'].abs()
data.loc[data['children'] == 20, 'children'] = 2
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

#### Замена типа данных столбца dob_years

Еще раз посмотрим общую информацию по таблице.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Тип данных для столбца days_employed был изменен ранее при обработке его пропущенных значений.
Из остальных столбцов тип данных надо заменить только для столбца dob_years, так как на данный момент значения этого столбца имеют вещественный тип данных. Переведем значения этого столбца в целочисленные.

In [ ]:
data['dob_years'] = data['dob_years'].astype('int')

Убедимся в том, что тип данных заменен.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


**Вывод**

- Используя метод astype() с аргументом (int), перевели значения столбцов dob_years и days_employed в целочисленные.
- Перевели все отрицательные значения столбца children в положительные, а также заменили артефакт 20 на 2 из предположения, что заполняющим это значение сотрудником имелось в виду значение 2 ребенка.

### Обработка дубликатов

#### Обработка столбца education

В столбце education содержатся дубликаты, отличающиеся регистром нескольких букв. Приведем все символы в каждой строке столбца education к нижнему регистру, используя метод str.lower(). 

In [ ]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

#### Обработка дубликатов

Подсчитаем количество дубликатов в массиве.

In [ ]:
data.duplicated().sum()

71

Удалим дубликаты и еще раз проверим число дубликатов в массиве.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

- Дубликаты в столбце education удалили с помощью метода str.lower(), так как они были вызваны отличием регистра букв. Значения в этом столбце заполняютс вручную. Чтобы избежать в дальнейшем появление таких дубликатов, необходимо применить вложенный список из значений уровня образования.
- Дубликаты во всей таблице удалили с помощью drop_duplicates.

### Лемматизация

Подсчитаем количество уникальных значений столбца purpose и выделим наиболее встречающиеся цели кредитов: недвижимость, образование, автомобиль и свадьба.

In [ ]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Импортируем библиотеку *pymystem3* и *collections*. Проведем лемматизацию столбца purpose и запишем результат в столбец purpose_lemma.

In [ ]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemma
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,взрослый,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослый,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,взрослый,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,взрослый,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослый,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,взрослый,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,взрослый,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,взрослый,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,взрослый,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,взрослый,"[покупка, , жилье, , для, , семья, \n]"


В каждой строке столбца purpose_lemma найдем ключевые слова из списка наиболее встречающихся целей кредитов и запишем в столбец purpose_new. Подсчитаем всевозможные категории целей кредитов.

In [ ]:
def set_purpose(purpose_lemma):
        if  'свадьба' in purpose_lemma:
                return 'свадьба'
        if 'автомобиль' in purpose_lemma:
                return 'автомобиль'
        if 'образование' in purpose_lemma:
                return 'образование'
        if ('недвижимость' in purpose_lemma) or ('жилье' in purpose_lemma):
                return 'недвижимость'
    
    
data['purpose_new'] = data['purpose_lemma'].apply(set_purpose)
data['purpose_new'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_new, dtype: int64

**Вывод**

В результате лемматизации установили наиболее встречающиеся цели кредитов: недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

Ранее была проведена категоризация по возрастному признаку и по целям кредита. Проведем категоризацию также по количеству имеющихся у клиента детей и по уровню его дохода.

#### Категоризация по количеству детей

In [ ]:
def children_group(children):
    
        if children == 0:
                return 'без детей'
        if children <= 2:
                return 'с 1 ребенком или 2 детьми'
        return 'многодетные'

    
data['children_group'] = data['children'].apply(children_group)
data['children_group'].value_counts()

без детей                    14091
с 1 ребенком или 2 детьми     6983
многодетные                    380
Name: children_group, dtype: int64

При категоризации клиентов по количеству имеющихся у них детей были выделены 3 группы:
- Клиенты без детей - составляют больше половины от общего количества клиентов банка
- Клиенты с 1 ребенком или 2 детьми
- Многодетные клиенты - клиенты с количеством детей от 3 и более

#### Категоризация по уровню дохода

In [ ]:
def total_income_group1(total_income):
          
        """
        Возвращает группу доходов по значению ежемесячного дохода total_income, используя правила:
        - 'Низкий доход при значении total_income менее или равно 60000
        - 'Средний доход' при значении total_income более 60000 и менее или равно 120000 
        - 'Средний доход' при значении total_income более 120000 и менее или равно 200000
        - 'Высокий доход' при значении total_income более 200000
        """
        
        if total_income <= 60000:
                return 'Низкий доход'
        if total_income <= 120000:
                return 'Средний доход'
        if total_income <= 200000:
                return 'Высокий средний доход'
        return 'Высокий доход'

    
data['total_income_group1'] = data['total_income'].apply(total_income_group1)
data['total_income_group1'].value_counts()

Высокий средний доход    9157
Средний доход            6425
Высокий доход            5066
Низкий доход              806
Name: total_income_group1, dtype: int64

При категоризации клиентов по уровню их ежемесячного дохода были выделены 4 группы:
- Клиенты с низким доходом менее 60000 в месяц
- Клиенты с средним доходом от 60000 и до 120000 в месяц
- Клиенты с высоким средним доходом от 120000 до 200000 в месяц
- Клиенты с средним доходом от 200000 в месяц и выше

**Вывод**

Для дальнейшего анализа была проведена категоризация клиентов по следующим признакам:
- возраст
- цель кредита
- количество детей
- уровень дохода

## Этап 3. Ответьте на вопросы

Для ответа на поставленные вопросы построим сводные таблицы, показывающие количество клиентов банка, сгруппированных по выбранной категории, а также по имевшейся у них задолженности по возврату кредита. Рассчитаем долю должников по каждой подкатегории и полученные значения запишем в новый стобец ratio. Сравним эти доли.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
 data_pivot_children = data.pivot_table(index = ['children_group'], columns = 'debt', values= 'children', aggfunc = 'count')
data_pivot_children['ratio'] = (data_pivot_children[1] / (data_pivot_children[1] + data_pivot_children[0]))*100
data_pivot_children.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
children_group,,,
с 1 ребенком или 2 детьми,6336,647,9.265359
многодетные,349,31,8.157895
без детей,13028,1063,7.543822


**Вывод**

Если у клиента есть дети, это увеличивает вероятность возникновения задолженности по кредиту.<br>
Воспитание детей требует больших расходов, часть бюджета клиента уходит на обеспечение их нужд, поэтому он скорее допустит просрочку по кредиту, чем клиент, не имеющий детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot_family_status = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'income_type', aggfunc = 'count')
data_pivot_family_status['ratio'] = ((data_pivot_family_status[1] / (data_pivot_family_status[1] + data_pivot_family_status[0]))*100)
data_pivot_family_status.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


**Вывод**

Клиенты, находящиеся в официальном браке или побывавшие в нем, гораздо ответственнее относятся к погашению задолженности по кредиту в срок. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_pivot_total_income = data.pivot_table(index = ['total_income_group1'], columns = 'debt', values = 'children', aggfunc = 'count')
data_pivot_total_income['ratio'] = ((data_pivot_total_income[1] / (data_pivot_total_income[1] + data_pivot_total_income[0]))*100)
data_pivot_total_income.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
total_income_group1,,,
Высокий средний доход,8360,797,8.703724
Средний доход,5888,537,8.357977
Высокий доход,4708,358,7.066719
Низкий доход,757,49,6.079404


**Вывод**

Анализ зависимости уровня дохода от возврата кредита в срок показывает, что наибольшее количество просрочек допускают клиенты со средним доходом. При этом клиенты с наименьшим уровнем доходов, а также с самым высоким доходом - это самые ответственные плательщики.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot_purpose = data.pivot_table(index = ['purpose_new'], columns = 'debt', values = 'children', aggfunc = 'count')
data_pivot_purpose['ratio'] = ((data_pivot_purpose[1] / (data_pivot_purpose[1] + data_pivot_purpose[0]))*100)
data_pivot_purpose.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
purpose_new,,,
автомобиль,3903,403,9.359034
образование,3643,370,9.220035
свадьба,2138,186,8.003442
недвижимость,10029,782,7.233373


**Вывод**

Самый низкий процент должников среди тех клиентов, кто берет кредит на недвижимость. Клиенты, взявшие кредит на свадьбу, возвращают их ответственее, чем те, кто брал кредит на автомобиль или получение образования. Хуже всего платят кредиты, взятые для образовательных целей и автокредиты.

## Шаг 4. Общий вывод

Изучив зависимости разных категорий клиентов от наличия у них задолженности, можно сделать такой вывод:
-  Наличие у клиента большого количества детей, воспитание которых требует значительных расходов из бюджета семьи, увеличивает возможность возникновения задолженности по возврату кредита.
- Люди в официальном браке, а также побывавшие в нем, ответственнее относятся к вовзрату кредита.
- С наименьшей вероятностью в должниках окажутся клиенты с высоким достатким, а также с низким достатком. Это можно объяснить тем. что люди, имеющие низкий достаток, ответственнее распоряжаются своим бюджетом.
- Самая ответственные плательщики - это клиенты, берущие кредиты на недвижимость.